In [ ]:
# -*- coding: utf-8 -*-
# @FileName: CircuitManusBase0509_FullRetry.ipynb
# @Version: Refactored - Async, Decorator Tools, Retry (Tool & Interaction), Diagram Entity, Log Saving, Enhanced Error Reporting
# @Author: Your Most Loyal & Dedicated Programmer (Refactored & Enhanced++)
# @Date: [Current Date]
# @License: MIT License
# @Description:
# ==============================================================================================
#  Manus 系统 技术实现说明 (全流程重试增强版)
# ==============================================================================================
#
# 尊敬的领导，这是我们为电路设计任务精心打造的异步 Agent 核心实现的再次增强版本。
# 在之前版本的基础上，本次最重要的增强是引入了 **整个用户请求处理流程的顶层重试机制**。
# 这意味着如果一次完整的交互（从感知到响应生成）由于 LLM 规划错误、无法解析的响应
# 或最终报告生成失败等原因而失败，系统将尝试从头开始重新处理用户的原始请求，
# 以期获得一次成功的交互。
#
# 核心逻辑依旧遵循业界标准的 Agentic 循环：感知 -> 规划 -> 行动 -> 观察 -> 响应生成。
#
# 系统的核心组件设计及增强点如下：
# 1.  `CircuitDiagram`: 专用电路图实体类，管理电路状态。
# 2.  `MemoryManager`: 集成 `CircuitDiagram`，管理短期、长期记忆及ID计数器。
# 3.  `LLMInterface`: 异步封装与 LLM 的交互。
# 4.  `OutputParser`: 解析 LLM 返回的文本，提取 `<think>` 和自定义 JSON 计划。
# 5.  `ToolExecutor`: (已有增强) 工具调用重试机制、详细错误报告、失败中止。
# 6.  内部工具 (Action Methods): (已有扩展) 扩展的工具集，利用 `CircuitDiagram`。
# 7.  `CircuitDesignAgent` - `process_user_request`: (核心增强)
#     -   **顶层交互重试机制**: 封装了对整个用户请求处理流程的重试逻辑。
#         如果一次完整的交互尝试失败（定义了明确的失败条件），Agent 会
#         重置部分短期记忆状态（恢复到当前用户请求刚进入时的状态）并重新尝试。
#     -   通过新的 `max_interaction_retries` 参数控制最大交互尝试次数。
#     -   增强了错误处理和用户反馈，以应对多次尝试失败的情况。
#
# 主要技术特性概述 (包括新增与原有)：
# -   **全面异步化 (`asyncio`)**
# -   **自定义 JSON 规划**
# -   **规划重试** (LLM 规划调用自身的重试)
# -   **工具执行重试** (单个工具执行的重试)
# -   **全流程交互重试** (新增: 整个用户请求处理流程的重试)
# -   **详细错误报告** (增强)
# -   **专用电路图实体 (`CircuitDiagram`)**
# -   **扩展工具集**
# -   **工具失败处理** (ToolExecutor 的失败中止)
# -   **记忆修剪**
# -   **动态工具注册**
# -   **本地日志保存**
#
# ==============================================================================================


# --- 基础库导入 ---
import re
import os
import json
import time
import logging
import sys
import asyncio
import traceback
import inspect # 用于在运行时发现通过装饰器注册的工具方法
import functools # 用于在装饰器中保留原始函数的重要元信息 (如名称、文档字符串)
from datetime import datetime # 用于生成带时间戳的日志文件名
from typing import List, Dict, Any, Optional, Tuple, Set, Union, Callable # Added Callable
import copy # For deep copying memory states for retry logic

from zhipuai import ZhipuAI # 引入与智谱 AI API 交互的官方 SDK

# --- 全局异步事件循环 ---
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# --- 日志系统配置 ---
log_directory = "logs"
if not os.path.exists(log_directory):
    os.makedirs(log_directory)
log_filename = os.path.join(log_directory, f"agent_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s [%(module)s.%(funcName)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stderr),
        logging.FileHandler(log_filename, encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)
logging.getLogger("zhipuai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

logger.info(f"日志将保存在: {log_filename}")

# --- 异步友好的打印函数 ---
async def async_print(message: str, end: str = '\n', flush: bool = True):
    sys.stdout.write(message + end)
    if flush:
        sys.stdout.flush()

# --- 电路元件数据类 ---
class CircuitComponent:
    __slots__ = ['id', 'type', 'value']
    def __init__(self, component_id: str, component_type: str, value: Optional[str] = None):
        if not isinstance(component_id, str) or not component_id.strip():
            raise ValueError("元件 ID 必须是有效的非空字符串")
        if not isinstance(component_type, str) or not component_type.strip():
            raise ValueError("元件类型必须是有效的非空字符串")
        self.id: str = component_id.strip().upper()
        self.type: str = component_type.strip()
        self.value: Optional[str] = str(value).strip() if value is not None and str(value).strip() else None
    def __str__(self) -> str:
        value_str = f" (值: {self.value})" if self.value else ""
        return f"元件: {self.type} (ID: {self.id}){value_str}"
    def __repr__(self) -> str:
        return f"CircuitComponent(id='{self.id}', type='{self.type}', value={repr(self.value)})"

# --- 电路图实体类 ---
class CircuitDiagram:
    def __init__(self):
        logger.info("[CircuitDiagram] 初始化电路图实体...")
        self.components: Dict[str, CircuitComponent] = {}
        self.connections: Set[Tuple[str, str]] = set()
        logger.info("[CircuitDiagram] 电路图实体初始化完成。")

    def add_component(self, component: CircuitComponent) -> bool:
        if component.id in self.components:
            logger.warning(f"[CircuitDiagram] 添加元件失败: ID '{component.id}' 已存在。")
            return False
        self.components[component.id] = component
        logger.info(f"[CircuitDiagram] 已添加元件: {component}")
        return True

    def get_component(self, component_id: str) -> Optional[CircuitComponent]:
        cid = component_id.strip().upper()
        return self.components.get(cid)

    def remove_component(self, component_id: str) -> bool:
        cid = component_id.strip().upper()
        if cid not in self.components:
            logger.warning(f"[CircuitDiagram] 移除元件失败: ID '{cid}' 不存在。")
            return False
        removed_component = self.components.pop(cid)
        logger.info(f"[CircuitDiagram] 已移除元件: {removed_component}")
        connections_to_remove = {conn for conn in self.connections if cid in conn}
        if connections_to_remove:
            self.connections -= connections_to_remove
            logger.info(f"[CircuitDiagram] 已移除与 '{cid}' 相关的 {len(connections_to_remove)} 个连接: {connections_to_remove}")
        return True

    def modify_component_value(self, component_id: str, new_value: Optional[str]) -> bool:
        cid = component_id.strip().upper()
        component = self.get_component(cid)
        if not component:
            logger.warning(f"[CircuitDiagram] 修改元件值失败: ID '{cid}' 不存在。")
            return False
        old_value = component.value
        processed_new_value = str(new_value).strip() if new_value is not None and str(new_value).strip() else None
        component.value = processed_new_value
        logger.info(f"[CircuitDiagram] 元件 '{cid}' 的值已从 '{old_value}' 修改为 '{component.value}'。")
        return True

    def add_connection(self, comp1_id: str, comp2_id: str) -> Tuple[bool, str]:
        id1 = comp1_id.strip().upper()
        id2 = comp2_id.strip().upper()
        if id1 == id2:
            msg = f"不能将元件 '{id1}' 连接到它自己。"
            logger.error(f"[CircuitDiagram] {msg}")
            return False, msg
        if id1 not in self.components:
            msg = f"无法连接：元件 '{id1}' 不存在。"
            logger.error(f"[CircuitDiagram] {msg}")
            return False, msg
        if id2 not in self.components:
            msg = f"无法连接：元件 '{id2}' 不存在。"
            logger.error(f"[CircuitDiagram] {msg}")
            return False, msg
        connection = tuple(sorted((id1, id2)))
        if connection in self.connections:
            msg = f"元件 '{id1}' 和 '{id2}' 之间已存在连接。"
            logger.info(f"[CircuitDiagram] {msg}")
            return True, msg
        self.connections.add(connection)
        logger.info(f"[CircuitDiagram] 已添加连接: {id1} <--> {id2}")
        return True, f"成功连接 '{id1}' 和 '{id2}'。"

    def remove_connection(self, comp1_id: str, comp2_id: str) -> Tuple[bool, str]:
        id1 = comp1_id.strip().upper()
        id2 = comp2_id.strip().upper()
        if id1 not in self.components:
            msg = f"无法移除连接：元件 '{id1}' 不存在。"
            logger.error(f"[CircuitDiagram] {msg}")
            return False, msg
        if id2 not in self.components:
            msg = f"无法移除连接：元件 '{id2}' 不存在。"
            logger.error(f"[CircuitDiagram] {msg}")
            return False, msg
        connection_to_remove = tuple(sorted((id1, id2)))
        if connection_to_remove not in self.connections:
            msg = f"元件 '{id1}' 和 '{id2}' 之间不存在连接。"
            logger.warning(f"[CircuitDiagram] {msg}")
            return False, msg
        self.connections.remove(connection_to_remove)
        logger.info(f"[CircuitDiagram] 已移除连接: {id1} <--> {id2}")
        return True, f"成功移除 '{id1}' 和 '{id2}' 之间的连接。"

    def get_text_description(self) -> str:
        logger.debug("[CircuitDiagram] 正在生成电路图文本描述...")
        num_components = len(self.components)
        num_connections = len(self.connections)
        if num_components == 0 and num_connections == 0:
            return "【当前电路图】: 电路为空。"
        desc_lines = ["【当前电路图】:"]
        desc_lines.append(f"  - 元件 ({num_components}):")
        if self.components:
            sorted_ids = sorted(self.components.keys())
            for cid in sorted_ids: desc_lines.append(f"    - {str(self.components[cid])}")
        else: desc_lines.append("    (无)")
        desc_lines.append(f"  - 连接 ({num_connections}):")
        if self.connections:
            sorted_connections = sorted(list(self.connections))
            for c1, c2 in sorted_connections: desc_lines.append(f"    - {c1} <--> {c2}")
        else: desc_lines.append("    (无)")
        description = "\n".join(desc_lines)
        logger.debug("[CircuitDiagram] 电路图文本描述生成完毕。")
        return description

    def clear(self):
        comp_count = len(self.components)
        conn_count = len(self.connections)
        self.components.clear()
        self.connections.clear()
        logger.info(f"[CircuitDiagram] 电路图已清空 (移除了 {comp_count} 个元件, {conn_count} 个连接)。")

# --- 工具注册装饰器 ---
def register_tool(description: str, parameters: Dict[str, Any]):
    def decorator(func: Callable) -> Callable: # Type hinting for func
        func._tool_schema = {"description": description, "parameters": parameters}
        func._is_tool = True
        @functools.wraps(func)
        def wrapper(*args: Any, **kwargs: Any) -> Any: # Type hinting for wrapper
            return func(*args, **kwargs)
        return wrapper
    return decorator

# --- 模块化组件：MemoryManager ---
class MemoryManager:
    def __init__(self, max_short_term_items: int = 20, max_long_term_items: int = 50):
        logger.info("[MemoryManager] 初始化记忆模块...")
        if max_short_term_items <= 1:
            raise ValueError("max_short_term_items 必须大于 1")
        self.max_short_term_items = max_short_term_items
        self.max_long_term_items = max_long_term_items
        self.short_term: List[Dict[str, Any]] = []
        self.long_term: List[str] = []
        self.circuit_diagram: CircuitDiagram = CircuitDiagram()
        self._component_counters: Dict[str, int] = {
            'R': 0, 'L': 0, 'B': 0, 'S': 0, 'C': 0, 'V': 0, 'G': 0, 'U': 0, 'O': 0,
            'I': 0, 'A': 0, 'D': 0, 'P': 0, 'F': 0, 'H': 0
        }
        logger.info(f"[MemoryManager] 记忆模块初始化完成。短期上限: {max_short_term_items} 条, 长期上限: {max_long_term_items} 条")

    def add_to_short_term(self, message: Dict[str, Any]):
        logger.debug(f"[MemoryManager] 添加消息到短期记忆 (Role: {message.get('role', 'N/A')}). 当前数量: {len(self.short_term)}")
        self.short_term.append(message)
        current_size = len(self.short_term)
        if current_size > self.max_short_term_items:
            logger.debug(f"[MemoryManager] 短期记忆超限 ({current_size}/{self.max_short_term_items})，执行修剪...")
            items_to_remove = current_size - self.max_short_term_items
            removed_count = 0
            indices_to_remove = []
            for i in range(len(self.short_term)):
                if self.short_term[i].get("role") != "system": # 保留 system prompt
                    indices_to_remove.append(i)
                if len(indices_to_remove) == items_to_remove:
                    break
            new_short_term = []
            indices_to_remove_set = set(indices_to_remove)
            removed_roles = []
            for i, item in enumerate(self.short_term):
                if i not in indices_to_remove_set:
                    new_short_term.append(item)
                else:
                    removed_roles.append(item.get('role', 'N/A'))
                    removed_count += 1
            self.short_term = new_short_term
            if removed_count > 0:
                 logger.info(f"[MemoryManager] 短期记忆修剪完成，移除了 {removed_count} 条最旧的非系统消息 (Roles: {removed_roles})。")
            else:
                 logger.warning("[MemoryManager] 短期记忆超限但未能找到足够的非系统消息进行移除。请检查 max_short_term_items 设置。")
        logger.debug(f"[MemoryManager] 添加后短期记忆数量: {len(self.short_term)}")

    def add_to_long_term(self, knowledge_snippet: str):
        logger.debug(f"[MemoryManager] 添加知识到长期记忆: '{knowledge_snippet[:100]}{'...' if len(knowledge_snippet) > 100 else ''}'. 当前数量: {len(self.long_term)}")
        self.long_term.append(knowledge_snippet)
        if len(self.long_term) > self.max_long_term_items:
            removed = self.long_term.pop(0)
            logger.info(f"[MemoryManager] 长期记忆超限 ({self.max_long_term_items}), 移除最旧知识: '{removed[:50]}...'")
        logger.debug(f"[MemoryManager] 添加后长期记忆数量: {len(self.long_term)}")

    def get_circuit_state_description(self) -> str:
        logger.debug("[MemoryManager] 正在请求 CircuitDiagram 生成电路状态描述...")
        return self.circuit_diagram.get_text_description()

    def get_memory_context_for_prompt(self, recent_long_term_count: int = 5) -> str:
        logger.debug("[MemoryManager] 正在格式化记忆上下文用于 Prompt...")
        circuit_desc = self.get_circuit_state_description()
        long_term_str = ""
        if self.long_term:
            actual_count = min(recent_long_term_count, len(self.long_term))
            if actual_count > 0:
                recent_items = self.long_term[-actual_count:]
                long_term_str = "\n\n【近期经验总结 (仅显示最近 N 条)】\n" + "\n".join(f"- {item}" for item in recent_items)
                logger.debug(f"[MemoryManager] 已提取最近 {len(recent_items)} 条长期记忆 (基础模式)。")
        long_term_str += "\n(注: 当前仅使用最近期记忆，未来版本将实现基于相关性的检索)"
        context = f"{circuit_desc}{long_term_str}".strip()
        logger.debug(f"[MemoryManager] 记忆上下文 (电路+长期) 格式化完成。")
        return context

    def generate_component_id(self, component_type: str) -> str:
        logger.debug(f"[MemoryManager] 正在为类型 '{component_type}' 生成唯一 ID...")
        type_map = {
            "resistor": "R", "电阻": "R", "capacitor": "C", "电容": "C",
            "battery": "B", "电池": "B", "voltage source": "V", "voltage": "V",
            "电压源": "V", "电压": "V", "led": "L", "发光二极管": "L", "switch": "S",
            "开关": "S", "ground": "G", "地": "G", "ic": "U", "chip": "U", "芯片": "U",
            "集成电路": "U", "inductor": "I", "电感": "I", "current source": "A",
            "电流源": "A", "diode": "D", "二极管": "D", "potentiometer": "P", "电位器": "P",
            "fuse": "F", "保险丝": "F", "header": "H", "排针": "H",
            "component": "O", "元件": "O",
        }
        for code in type_map.values():
            if code not in self._component_counters:
                 self._component_counters[code] = 0
        cleaned_type = component_type.strip().lower()
        type_code = "O"
        best_match_len = 0
        for keyword, code in type_map.items():
            if keyword in cleaned_type and len(keyword) > best_match_len:
                type_code = code
                best_match_len = len(keyword)
        if type_code == "O" and cleaned_type not in ["component", "元件"]:
             logger.warning(f"[MemoryManager] 未找到类型 '{component_type}' 的特定前缀，将使用通用前缀 'O'。")
        MAX_ID_ATTEMPTS = 100
        for attempt in range(MAX_ID_ATTEMPTS):
            self._component_counters[type_code] += 1
            gen_id = f"{type_code}{self._component_counters[type_code]}"
            if not self.circuit_diagram.get_component(gen_id):
                logger.debug(f"[MemoryManager] 生成唯一 ID: '{gen_id}' (尝试 {attempt + 1})")
                return gen_id
            logger.warning(f"[MemoryManager] ID '{gen_id}' 已存在（可能是手动添加的），尝试下一个。")
        # 如果ID生成后仍然存在，说明计数器可能与实际元件ID不同步，需要回退计数器
        self._component_counters[type_code] -= MAX_ID_ATTEMPTS 
        if self._component_counters[type_code] < 0: self._component_counters[type_code] = 0
        raise RuntimeError(f"未能为类型 '{component_type}' (代码 '{type_code}') 生成唯一 ID ({MAX_ID_ATTEMPTS} 次尝试后)。")

    def clear_circuit(self):
        logger.info("[MemoryManager] 正在清空电路状态...")
        self.circuit_diagram.clear()
        self._component_counters = {k: 0 for k in self._component_counters}
        logger.info(f"[MemoryManager] 电路状态已清空 (电路图清空，ID 计数器重置)。")

# --- 模块化组件：LLMInterface ---
class LLMInterface:
    def __init__(self, api_key: str, model_name: str = "glm-4-flash", default_temperature: float = 0.1, default_max_tokens: int = 4095):
        logger.info(f"[LLMInterface] 初始化 LLM 接口，目标模型: {model_name}")
        if not api_key: raise ValueError("智谱 AI API Key 不能为空")
        try:
            self.client = ZhipuAI(api_key=api_key)
            logger.info("[LLMInterface] 智谱 AI 客户端初始化成功。")
        except Exception as e:
            logger.critical(f"[LLMInterface] 初始化智谱 AI 客户端失败: {e}", exc_info=True)
            raise ConnectionError(f"初始化智谱 AI 客户端失败: {e}") from e
        self.model_name = model_name
        self.default_temperature = default_temperature
        self.default_max_tokens = default_max_tokens
        logger.info(f"[LLMInterface] LLM 接口初始化完成 (Model: {model_name}, Temp: {default_temperature}, MaxTokens: {default_max_tokens})。")

    async def call_llm(self, messages: List[Dict[str, Any]], use_tools: bool = False, tool_choice: Optional[str] = None) -> Any:
        call_args = {
            "model": self.model_name,
            "messages": messages,
            "temperature": self.default_temperature,
            "max_tokens": self.default_max_tokens,
        }
        if use_tools:
            logger.warning("[LLMInterface] 注意: 请求使用 SDK tools 参数。")
        else:
            logger.info(f"[LLMInterface] 准备异步调用 LLM ({self.model_name}，自定义 JSON/无内置工具模式)...")
        logger.debug(f"[LLMInterface] 发送的消息条数: {len(messages)}")
        try:
            start_time = time.monotonic()
            response = await asyncio.to_thread(
                self.client.chat.completions.create,
                **call_args
            )
            duration = time.monotonic() - start_time
            logger.info(f"[LLMInterface] LLM 异步调用成功。耗时: {duration:.3f} 秒。")
            if response:
                if response.usage:
                    logger.info(f"[LLMInterface] Token 统计: Prompt={response.usage.prompt_tokens}, Completion={response.usage.completion_tokens}, Total={response.usage.total_tokens}")
                if response.choices:
                    logger.info(f"[LLMInterface] 完成原因: {response.choices[0].finish_reason}")
                else:
                     logger.warning("[LLMInterface] LLM 响应中缺少 'choices' 字段。")
            else:
                 logger.error("[LLMInterface] LLM API 调用返回了 None！")
                 raise ConnectionError("LLM API call returned None.")
            return response
        except Exception as e:
            logger.error(f"[LLMInterface] LLM API 异步调用失败: {e}", exc_info=True)
            raise

# --- 模块化组件：OutputParser ---
class OutputParser:
    def __init__(self):
        logger.info("[OutputParser] 初始化输出解析器 (用于自定义 JSON 和文本解析)。")

    def parse_planning_response(self, response_message: Any) -> Tuple[str, Optional[Dict[str, Any]], str]:
        logger.debug("[OutputParser] 开始解析规划响应 (自定义 JSON 模式)...")
        thinking_process = "未能提取思考过程。"
        plan = None
        error_message = ""
        if response_message is None:
            error_message = "LLM 响应对象为 None。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message
        raw_content = getattr(response_message, 'content', None)
        if not raw_content or not raw_content.strip():
            tool_calls = getattr(response_message, 'tool_calls', None)
            error_message = "LLM 响应内容为空或仅包含空白字符。"
            if tool_calls: error_message = "LLM 响应内容为空，但意外地包含了 tool_calls。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message
        think_match = re.search(r'<think>(.*?)</think>', raw_content, re.IGNORECASE | re.DOTALL)
        json_part_start_index = 0
        if think_match:
            thinking_process = think_match.group(1).strip()
            json_part_start_index = think_match.end()
            logger.debug("[OutputParser] 成功提取 <think> 内容。")
        else:
            thinking_process = "警告：未找到 <think> 标签，将尝试解析后续内容为 JSON。"
            logger.warning(f"[OutputParser] {thinking_process}")
        potential_json_part = raw_content[json_part_start_index:].strip()
        logger.debug(f"[OutputParser] 提取出的待解析 JSON 字符串 (前 1000 字符): >>>\n{potential_json_part[:1000]}{'...' if len(potential_json_part) > 1000 else ''}\n<<<")
        if not potential_json_part:
            error_message = "提取出的潜在 JSON 内容为空。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message
        final_json_string = ""
        try:
            json_string_to_parse = potential_json_part
            if json_string_to_parse.startswith("```json"): json_string_to_parse = json_string_to_parse[len("```json"):].strip()
            if json_string_to_parse.startswith("```"): json_string_to_parse = json_string_to_parse[len("```"):].strip()
            if json_string_to_parse.endswith("```"): json_string_to_parse = json_string_to_parse[:-len("```")].strip()
            json_start, json_end = -1, -1
            brace_level, square_level, in_string = 0, 0, False
            string_char, possible_start = '', -1
            first_brace = json_string_to_parse.find('{')
            first_square = json_string_to_parse.find('[')
            if first_brace != -1 and (first_square == -1 or first_brace < first_square): possible_start = first_brace
            elif first_square != -1 and (first_brace == -1 or first_square < first_brace): possible_start = first_square
            if possible_start == -1: raise json.JSONDecodeError("无法在文本中定位 JSON 对象或数组的起始。", json_string_to_parse, 0)
            json_start = possible_start
            start_char = json_string_to_parse[json_start]
            for i in range(json_start, len(json_string_to_parse)):
                char = json_string_to_parse[i]
                if in_string:
                    if char == string_char and (i == json_start or json_string_to_parse[i-1] != '\\'): in_string = False
                else:
                    if char == '"' or char == "'": in_string = True; string_char = char
                    elif char == '{' and start_char == '{': brace_level += 1
                    elif char == '}' and start_char == '{': brace_level -= 1
                    elif char == '[' and start_char == '[': square_level += 1
                    elif char == ']' and start_char == '[': square_level -= 1
                if not in_string and ((start_char == '{' and brace_level == 0 and char == '}') or \
                   (start_char == '[' and square_level == 0 and char == ']')):
                    json_end = i + 1; break
            if json_end == -1: raise json.JSONDecodeError("无法在文本中找到匹配的 JSON 结束符。", json_string_to_parse, len(json_string_to_parse)-1)
            final_json_string = json_string_to_parse[json_start:json_end]
            logger.debug(f"[OutputParser] 精准提取的 JSON 字符串: >>>\n{final_json_string}\n<<<")
            parsed_json = json.loads(final_json_string)
            logger.debug("[OutputParser] JSON 字符串解析成功。")
            if not isinstance(parsed_json, dict): raise ValueError("解析结果不是一个 JSON 对象 (字典)。")
            if "is_tool_calls" not in parsed_json or not isinstance(parsed_json["is_tool_calls"], bool): raise ValueError("JSON 对象缺少必需的布尔字段 'is_tool_calls'。")
            tool_list = parsed_json.get("tool_list")
            if parsed_json["is_tool_calls"]:
                if not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 字段必须是一个列表。")
                if not tool_list: raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 列表不能为空。")
                indices_set = set()
                for i, tool_item in enumerate(tool_list):
                    if not isinstance(tool_item, dict): raise ValueError(f"'tool_list' 中索引 {i} 的元素不是字典。")
                    if not tool_item.get("toolname") or not isinstance(tool_item["toolname"], str): raise ValueError(f"'tool_list' 中索引 {i} 缺少有效的 'toolname' 字符串。")
                    if "params" not in tool_item or not isinstance(tool_item["params"], dict): raise ValueError(f"'tool_list' 中索引 {i} 缺少 'params' 字典。")
                    if not tool_item.get("index") or not isinstance(tool_item["index"], int) or tool_item["index"] <= 0: raise ValueError(f"'tool_list' 中索引 {i} 缺少有效正整数 'index'。")
                    if tool_item['index'] in indices_set: raise ValueError(f"'tool_list' 中索引 {i} 的 'index' 值 {tool_item['index']} 与之前的重复。")
                    indices_set.add(tool_item['index'])
                max_index = max(indices_set) if indices_set else 0
                if len(indices_set) != max_index or set(range(1, max_index + 1)) != indices_set:
                     logger.warning(f"[OutputParser] 验证警告: 'tool_list' 中的 'index' ({sorted(list(indices_set))}) 不连续或不从 1 开始。")
            else:
                if tool_list is not None and not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 字段必须是 null 或列表。")
                if isinstance(tool_list, list) and tool_list: raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 必须为空列表 [] 或 null。")
            direct_reply = parsed_json.get("direct_reply")
            if not parsed_json["is_tool_calls"]:
                if not isinstance(direct_reply, str) or not direct_reply.strip():
                    raise ValueError("当 'is_tool_calls' 为 false 时, 必须提供有效的非空 'direct_reply' 字符串。")
            else:
                if direct_reply is not None and not isinstance(direct_reply, str): raise ValueError("当 'is_tool_calls' 为 true 时, 'direct_reply' 字段必须是 null 或字符串。")
            plan = parsed_json
            logger.info("[OutputParser] 自定义 JSON 计划解析和验证成功！")
        except json.JSONDecodeError as json_err:
            error_message = f"解析 JSON 失败: {json_err}。Raw JSON (截断): '{potential_json_part[:200]}...'"
            logger.error(f"[OutputParser] JSON 解析失败: {error_message}")
        except ValueError as validation_err:
            error_message = f"JSON 结构验证失败: {validation_err}。"
            logger.error(f"[OutputParser] JSON 结构验证失败: {error_message} JSON content: {final_json_string if final_json_string else potential_json_part[:200]}")
        except Exception as e:
            error_message = f"解析规划响应时发生未知错误: {e}"
            logger.error(f"[OutputParser] 解析时未知错误: {error_message}", exc_info=True)
        return thinking_process, plan, error_message

    def _parse_llm_text_content(self, text_content: str) -> Tuple[str, str]:
        logger.debug("[OutputParser._parse_llm_text_content] 正在解析最终文本内容...")
        if not text_content: return "思考过程为空。", "回复内容为空。"
        thinking_process = "未能提取思考过程。"
        formal_reply = text_content.strip()
        # 增强对最终回复中意外JSON的检测 (针对老板的反馈)
        # 尝试找到<think>块之后，但在任何意外的JSON块之前的内容作为正式回复
        think_match = re.search(r'<think>(.*?)</think>', text_content, re.IGNORECASE | re.DOTALL)
        content_after_think = text_content
        if think_match:
            thinking_process = think_match.group(1).strip()
            content_after_think = text_content[think_match.end():].lstrip() # lstrip to remove leading newlines/spaces
            content_before_think = text_content[:think_match.start()].strip()
            if content_before_think:
                logger.warning(f"[OutputParser._parse_llm_text_content] 在 <think> 标签之前检测到非空白内容: '{content_before_think[:50]}...'。这部分内容已被忽略。")
        else:
            logger.warning("[OutputParser._parse_llm_text_content] 未找到 <think>...</think> 标签。")
            thinking_process = "未能提取思考过程 - LLM 可能未按预期包含<think>标签。"

        # 检查 content_after_think 是否包含意外的 JSON 对象
        # 这是一个启发式检查，查找以 `{` 开头并包含典型规划关键字的块
        json_like_match = re.search(r'^\s*\{.*("is_tool_calls"|"tool_list").*\}', content_after_think, re.DOTALL | re.MULTILINE)
        if json_like_match:
            logger.warning(f"[OutputParser._parse_llm_text_content] 最终回复中检测到意外的 JSON 结构！将尝试提取 JSON 之前的内容作为正式回复。JSON片段: {json_like_match.group(0)[:100]}...")
            formal_reply = content_after_think[:json_like_match.start()].strip()
            if not formal_reply: # 如果JSON之前没有文本
                formal_reply = "智能助理的最终报告格式不正确（包含了意外的JSON计划），无法显示完整回复。"
                logger.error("[OutputParser._parse_llm_text_content] 意外的JSON结构之前没有有效的回复文本。")
        else:
            formal_reply = content_after_think.strip()

        thinking_process = thinking_process if thinking_process else "提取的思考过程为空白。"
        formal_reply = formal_reply if formal_reply else "LLM 未生成最终报告内容，或报告格式不正确。"
        logger.debug(f"[OutputParser._parse_llm_text_content] 解析结果 - 思考长度: {len(thinking_process)}, 回复长度: {len(formal_reply)}")
        return thinking_process, formal_reply

# --- 模块化组件：ToolExecutor ---
class ToolExecutor:
    def __init__(self, agent_instance: 'CircuitDesignAgent', max_retries: int = 3, retry_delay_seconds: float = 1.0):
        logger.info(f"[ToolExecutor] 初始化工具执行器 (异步, 失败中止, 最多重试 {max_retries-1} 次, 延迟 {retry_delay_seconds}s)。")
        if not isinstance(agent_instance, CircuitDesignAgent):
            raise TypeError("ToolExecutor 需要一个 CircuitDesignAgent 实例。")
        self.agent_instance = agent_instance
        if not hasattr(agent_instance, 'memory_manager') or not isinstance(agent_instance.memory_manager, MemoryManager):
            raise TypeError("Agent 实例缺少有效的 MemoryManager。")
        self.max_attempts = max(1, max_retries)
        self.retry_delay_seconds = retry_delay_seconds
        logger.info(f"[ToolExecutor] 工具调用将尝试最多 {self.max_attempts} 次。")

    async def execute_tool_calls(self, mock_tool_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        logger.info(f"[ToolExecutor] 准备异步执行最多 {len(mock_tool_calls)} 个工具调用 (按顺序，失败中止，带重试)...")
        execution_results = []
        if not mock_tool_calls:
            logger.info("[ToolExecutor] 没有工具需要执行。")
            return []
        total_tools_planned = len(mock_tool_calls)
        for i, mock_call in enumerate(mock_tool_calls):
            current_tool_index_in_plan = i + 1
            function_name = "unknown_function"
            tool_call_id = mock_call.get('id', f'mock_id_{i}')
            arguments = {}
            tool_display_name = "未知工具"
            action_result = None # Initialize action_result for each tool call
            try:
                func_info = mock_call.get('function')
                if not isinstance(func_info, dict) or 'name' not in func_info or 'arguments' not in func_info:
                    err_msg = f"模拟 ToolCall 对象结构无效 (ID: {tool_call_id}). 对象: {mock_call}"
                    logger.error(f"[ToolExecutor] {err_msg}")
                    action_result = {"status": "failure", "message": "错误: 内部工具调用结构无效。", "error": {"type": "MalformedMockCall", "details": err_msg}}
                    # execution_results.append({"tool_call_id": tool_call_id, "result": action_result}) # Appended later
                    await async_print(f"  ❌ [{current_tool_index_in_plan}/{total_tools_planned}] 内部错误: 工具调用结构无效。")
                    # break # Break is handled after appending result
                else: # func_info is valid
                    function_name = func_info['name']
                    function_args_str = func_info['arguments']
                    tool_display_name = function_name.replace('_tool', '').replace('_', ' ').title()
                    logger.info(f"[ToolExecutor] 处理工具调用 {current_tool_index_in_plan}/{total_tools_planned}: Name='{function_name}', MockID='{tool_call_id}'")
                    logger.debug(f"[ToolExecutor] 参数 JSON 字符串: '{function_args_str}'")
                    await async_print(f"  [{current_tool_index_in_plan}/{total_tools_planned}] 准备执行: {tool_display_name}...")
                    try:
                        arguments = json.loads(function_args_str) if function_args_str else {}
                        if not isinstance(arguments, dict): raise TypeError("参数必须是 JSON 对象 (字典)")
                        logger.debug(f"[ToolExecutor] 参数解析成功: {arguments}")
                    except (json.JSONDecodeError, TypeError) as json_err:
                        err_msg = f"工具 '{function_name}' (ID: {tool_call_id}) 的参数 JSON 解析失败: {json_err}. Raw: '{function_args_str}'"
                        logger.error(f"[ToolExecutor] 参数解析错误: {err_msg}", exc_info=True)
                        action_result = {"status": "failure", "message": f"错误: 工具 '{tool_display_name}' 的参数格式错误。", "error": {"type": "ArgumentParsingError", "details": err_msg}}
                        await async_print(f"  ❌ [{current_tool_index_in_plan}/{total_tools_planned}] 操作失败: {tool_display_name}. 错误: 参数解析失败。")
                        # execution_results.append({"tool_call_id": tool_call_id, "result": action_result}) # Appended later
                        # break # Break is handled after appending result
                    
                    if action_result is None: # If no parsing error, proceed to execute
                        tool_action_method = getattr(self.agent_instance, function_name, None)
                        if not callable(tool_action_method):
                            err_msg = f"Agent 未实现名为 '{function_name}' 的工具方法 (ID: {tool_call_id})。"
                            logger.error(f"[ToolExecutor] 工具未实现: {err_msg}")
                            action_result = {"status": "failure", "message": f"错误: {err_msg}", "error": {"type": "ToolNotFoundError", "details": f"Action method '{function_name}' not found on agent."}}
                            await async_print(f"  ❌ [{current_tool_index_in_plan}/{total_tools_planned}] 操作失败: {tool_display_name}. 错误: 工具未实现。")
                            # execution_results.append({"tool_call_id": tool_call_id, "result": action_result}) # Appended later
                            # break # Break is handled after appending result
                        else: # Tool method exists, attempt execution with retries
                            for attempt in range(self.max_attempts):
                                current_attempt = attempt + 1
                                logger.info(f"[ToolExecutor] >>> 正在调用 Action 方法: '{function_name}' (ID: {tool_call_id}), 尝试 {current_attempt}/{self.max_attempts}")
                                logger.debug(f"[ToolExecutor] 传递给 '{function_name}' (尝试 {current_attempt}) 的参数: {arguments}")
                                await async_print(f"  [{current_tool_index_in_plan}/{total_tools_planned}] ⚙️ 正在执行 '{tool_display_name}' (尝试 {current_attempt}/{self.max_attempts})...")
                                try:
                                    action_result_current_attempt = await asyncio.to_thread(tool_action_method, arguments=arguments)
                                    if not isinstance(action_result_current_attempt, dict) or 'status' not in action_result_current_attempt or 'message' not in action_result_current_attempt:
                                        err_msg = f"Action '{function_name}' (ID: {tool_call_id}, 尝试 {current_attempt}) 返回结构无效: {str(action_result_current_attempt)[:200]}..."
                                        logger.error(f"[ToolExecutor] Action 返回结构错误: {err_msg}")
                                        action_result = {"status": "failure", "message": f"错误: 工具 '{tool_display_name}' 返回结果结构无效。", "error": {"type": "InvalidActionResult", "details": err_msg, "attempt": current_attempt}}
                                    else:
                                        logger.info(f"[ToolExecutor] Action '{function_name}' (ID: {tool_call_id}, 尝试 {current_attempt}) 执行完毕。状态: {action_result_current_attempt.get('status', 'N/A')}")
                                        action_result = action_result_current_attempt # Store current attempt result
                                    if action_result.get("status") == "success":
                                        break # Success, break retry loop
                                    if "error" not in action_result:
                                        action_result["error"] = {"type": "GenericToolFailure", "details": "Tool reported failure without specific error info.", "attempt": current_attempt}
                                    elif isinstance(action_result["error"], dict):
                                        action_result["error"]["attempt"] = current_attempt
                                    else:
                                        action_result["error"] = {"type": "MalformedErrorField", "details": str(action_result["error"]), "attempt": current_attempt}
                                except TypeError as te:
                                    err_msg = f"调用 Action '{function_name}' (ID: {tool_call_id}, 尝试 {current_attempt}) 时参数不匹配: {te}. 传入参数: {arguments}"
                                    logger.error(f"[ToolExecutor] Action 调用参数错误: {err_msg}", exc_info=True)
                                    action_result = {"status": "failure", "message": f"错误: 调用工具 '{tool_display_name}' 时参数错误。", "error": {"type": "ArgumentMismatchError", "details": err_msg, "attempt": current_attempt}}
                                except Exception as exec_err:
                                    err_msg = f"Action '{function_name}' (ID: {tool_call_id}, 尝试 {current_attempt}) 执行期间意外错误: {exec_err}"
                                    logger.error(f"[ToolExecutor] Action 执行内部错误: {err_msg}", exc_info=True)
                                    action_result = {"status": "failure", "message": f"错误: 执行工具 '{tool_display_name}' 时发生内部错误。", "error": {"type": "ToolExecutionError", "details": str(exec_err), "traceback": traceback.format_exc(limit=3), "attempt": current_attempt}}
                                if current_attempt == self.max_attempts or action_result.get("status") == "success":
                                    break 
                                else:
                                    logger.warning(f"[ToolExecutor] Action '{function_name}' (ID: {tool_call_id}) 尝试 {current_attempt} 失败。将在 {self.retry_delay_seconds} 秒后重试...")
                                    await async_print(f"    ⚠️ '{tool_display_name}' 执行失败 (尝试 {current_attempt}), 将在 {self.retry_delay_seconds}s 后重试...")
                                    await asyncio.sleep(self.retry_delay_seconds)
                            # After retry loop, action_result holds the final result of attempts
                            status_icon = "✅" if action_result.get("status") == "success" else "❌"
                            msg_preview = action_result.get('message', '无消息')[:80] + ('...' if len(action_result.get('message', '')) > 80 else '')
                            final_attempt_count = action_result.get("error", {}).get("attempt", current_attempt if action_result.get("status") == "success" else self.max_attempts)
                            if action_result.get("status") == "success":
                                await async_print(f"  {status_icon} [{current_tool_index_in_plan}/{total_tools_planned}] 操作完成: {tool_display_name} (尝试 {final_attempt_count}). 结果: {msg_preview}")
                            else:
                                await async_print(f"  {status_icon} [{current_tool_index_in_plan}/{total_tools_planned}] 操作失败: {tool_display_name} (共尝试 {final_attempt_count} 次). 最终结果: {msg_preview}")
                                if "error" in action_result and isinstance(action_result["error"], dict):
                                    logger.error(f"[ToolExecutor] 工具 '{function_name}' (ID: {tool_call_id}) 最终失败详情: Type={action_result['error'].get('type')}, Details={action_result['error'].get('details')}")
            except Exception as outer_err:
                 err_msg = f"处理工具调用 '{function_name}' (Mock ID: {tool_call_id}) 时发生顶层意外错误: {outer_err}"
                 logger.error(f"[ToolExecutor] 处理工具调用时顶层错误: {err_msg}", exc_info=True)
                 action_result = {"status": "failure", "message": f"错误: 处理工具 '{tool_display_name or function_name}' 时发生未知内部错误。", "error": {"type": "UnexpectedExecutorError", "details": str(outer_err), "traceback": traceback.format_exc(limit=3)}}
                 await async_print(f"  ❌ [{current_tool_index_in_plan}/{total_tools_planned}] 操作失败: {tool_display_name or function_name}. 错误: 未知内部错误。")
            
            if action_result is None: # Should be assigned in all paths
                 logger.error(f"[ToolExecutor] 内部逻辑错误: 工具 '{function_name}' (Mock ID: {tool_call_id}) 未生成任何结果。标记为失败。")
                 action_result = {"status": "failure", "message": f"错误: 工具 '{tool_display_name}' 未返回结果。", "error": {"type": "MissingResultError", "details": "Execution pipeline failed to produce a result."}}
            execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
            logger.debug(f"[ToolExecutor] 已记录工具 '{tool_call_id}' 的执行结果。")
            if action_result.get("status") != "success":
                logger.warning(f"[ToolExecutor] 工具 '{function_name}' (Mock ID: {tool_call_id}) 执行最终失败 (状态: {action_result.get('status', 'failure')})。中止后续工具执行。")
                await async_print(f"  ⚠️ 由于工具 '{tool_display_name}' 执行失败，后续计划已中止。")
                break
        total_executed_or_attempted = len(execution_results)
        logger.info(f"[ToolExecutor] 所有 {total_executed_or_attempted}/{total_tools_planned} 个计划工具调用处理完毕 (可能因失败提前中止)。")
        return execution_results

# --- Agent 核心类 ---
class CircuitDesignAgent:
    def __init__(self, api_key: str, model_name: str = "glm-4-flash",
                 max_short_term_items: int = 25, max_long_term_items: int = 50,
                 planning_llm_retries: int = 1, 
                 tool_executor_retries: int = 3, # Default 3 total attempts for tools
                 max_interaction_retries: int = 1 # Default 1 retry for the whole interaction (total 2 attempts)
                ):
        logger.info(f"\n{'='*30} Agent 初始化开始 (全流程重试增强版) {'='*30}")
        logger.info("[Agent Init] 正在启动电路设计助理 (全流程重试增强版)...")
        try:
            self.memory_manager = MemoryManager(max_short_term_items, max_long_term_items)
            self.llm_interface = LLMInterface(api_key=api_key, model_name=model_name)
            self.output_parser = OutputParser()
            self.tool_executor = ToolExecutor(
                agent_instance=self, 
                max_retries=tool_executor_retries,
                retry_delay_seconds=1.5
            ) 
        except (ValueError, ConnectionError, TypeError) as e:
            logger.critical(f"[Agent Init] 核心模块初始化失败: {e}", exc_info=True)
            sys.stderr.write(f"\n🔴 Agent 核心模块初始化失败: {e}\n请检查配置或依赖！程序无法启动。\n")
            sys.stderr.flush()
            sys.exit(1)
        self.planning_llm_retries = max(0, planning_llm_retries) # Retries for planning LLM call itself
        self.max_interaction_attempts = max(1, max_interaction_retries + 1) # Total attempts for the whole interaction
        logger.info(f"[Agent Init] 规划 LLM 调用失败时将重试 {self.planning_llm_retries} 次。")
        logger.info(f"[Agent Init] 工具执行失败时将尝试最多 {tool_executor_retries} 次。")
        logger.info(f"[Agent Init] 整个用户请求处理流程将尝试最多 {self.max_interaction_attempts} 次。")

        self.tools_registry: Dict[str, Dict[str, Any]] = {}
        logger.info("[Agent Init] 正在动态发现并注册已标记的工具...")
        for name, method in inspect.getmembers(self, predicate=inspect.ismethod):
            if hasattr(method, '_is_tool') and method._is_tool:
                schema = getattr(method, '_tool_schema', None)
                if schema and isinstance(schema, dict) and 'description' in schema and 'parameters' in schema:
                    self.tools_registry[name] = schema
                    logger.info(f"[Agent Init] ✓ 已注册工具: '{name}'")
                else:
                    logger.warning(f"[Agent Init] 发现工具 '{name}' 但其 Schema 结构不完整或无效，已跳过。Schema: {schema}")
        if not self.tools_registry:
            logger.warning("[Agent Init] 未发现任何通过 @register_tool 注册的工具！")
        else:
            logger.info(f"[Agent Init] 共发现并注册了 {len(self.tools_registry)} 个工具。")
            logger.debug(f"[Agent Init] 工具注册表详情:\n{json.dumps(self.tools_registry, indent=2, ensure_ascii=False)}")
        logger.info(f"\n{'='*30} Agent 初始化成功 (全流程重试增强版) {'='*30}\n")
        print("我是电路设计编程助理 (全流程重试增强版)！")
        print("已准备好接收指令。特性: 异步核心, 动态工具注册, 多级重试机制, 电路图实体, 日志保存。")
        print("-" * 70)
        sys.stdout.flush()

    # --- Action Implementations (Interacting with CircuitDiagram) ---
    @register_tool(
        description="添加一个新的电路元件 (如电阻, 电容, 电池, LED, 开关, 芯片, 地线等)。如果用户未指定 ID，我会自动生成。元件值是可选的。",
        parameters={
            "type": "object", "properties": {
                "component_type": {"type": "string", "description": "元件的类型 (例如: '电阻', 'LED', '9V 电池')."},
                "component_id": {"type": "string", "description": "可选的用户指定 ID。如果省略会自动生成。"},
                "value": {"type": "string", "description": "可选的元件值 (例如: '1k', '10uF', '9V')."}
            }, "required": ["component_type"]
        }
    )
    def add_component_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: AddComponent] 执行添加元件操作。")
        logger.debug(f"[Action: AddComponent] 收到参数: {arguments}")
        component_type = arguments.get("component_type")
        component_id_req = arguments.get("component_id")
        value = arguments.get("value")
        if not component_type or not isinstance(component_type, str) or not component_type.strip():
            msg="元件类型是必需的，并且必须是有效的字符串。"
            logger.error(f"[Action: AddComponent] 输入验证失败: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}
        target_id = None; id_was_generated = False; user_provided_id_validated = None
        if component_id_req and isinstance(component_id_req, str) and component_id_req.strip():
            user_provided_id = component_id_req.strip().upper()
            if re.match(r'^[a-zA-Z0-9_][a-zA-Z0-9_-]*$', user_provided_id):
                if self.memory_manager.circuit_diagram.get_component(user_provided_id):
                    msg=f"元件 ID '{user_provided_id}' 已被占用。"
                    logger.error(f"[Action: AddComponent] ID 冲突: {msg}")
                    return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDConflict", "details": msg}}
                target_id = user_provided_id; user_provided_id_validated = target_id
            else: logger.warning(f"[Action: AddComponent] 用户提供的 ID '{component_id_req}' 格式无效, 将自动生成。")
        if target_id is None:
            try:
                target_id = self.memory_manager.generate_component_id(component_type)
                id_was_generated = True
            except RuntimeError as e:
                msg=f"无法自动为类型 '{component_type}' 生成唯一 ID: {e}"
                logger.error(f"[Action: AddComponent] ID 生成失败: {msg}", exc_info=True)
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDGenerationFailed", "details": str(e)}}
        processed_value = str(value).strip() if value is not None and str(value).strip() else None
        try:
            if target_id is None: raise ValueError("内部错误：未能最终确定元件 ID。")
            new_component = CircuitComponent(target_id, component_type, processed_value)
            if self.memory_manager.circuit_diagram.add_component(new_component):
                logger.info(f"[Action: AddComponent] 成功添加元件 '{new_component.id}' 到电路图。")
                success_message = f"操作成功: 已添加元件 {str(new_component)}。"
                if id_was_generated: success_message += f" (系统自动分配 ID '{new_component.id}')"
                elif user_provided_id_validated: success_message += f" (使用了您指定的 ID '{user_provided_id_validated}')"
                self.memory_manager.add_to_long_term(f"添加了元件: {str(new_component)}")
                return {"status": "success", "message": success_message, "data": {"id": new_component.id, "type": new_component.type, "value": new_component.value}}
            else:
                msg = f"添加元件 '{new_component.id}' 失败，ID 可能已存在。"
                logger.error(f"[Action: AddComponent] {msg}")
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "AddComponentFailed", "details": "Failed to add component to diagram, ID conflict likely."}}
        except ValueError as ve: # From CircuitComponent constructor
            msg=f"创建元件对象时发生验证错误: {ve}"
            logger.error(f"[Action: AddComponent] 元件创建错误: {msg}", exc_info=True)
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentValidationError", "details": str(ve)}}
        except Exception as e:
            msg=f"添加元件时发生未知的内部错误: {e}"
            logger.error(f"[Action: AddComponent] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 添加元件时发生未知内部错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="使用两个已存在元件的 ID 将它们连接起来。",
        parameters={
            "type": "object", "properties": {
                "comp1_id": {"type": "string", "description": "第一个元件的 ID。"},
                "comp2_id": {"type": "string", "description": "第二个元件的 ID。"}
            }, "required": ["comp1_id", "comp2_id"]
        }
    )
    def connect_components_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: ConnectComponents] 执行连接元件操作。")
        comp1_id = arguments.get("comp1_id")
        comp2_id = arguments.get("comp2_id")
        if not comp1_id or not comp2_id:
            msg = "必须提供两个有效的元件 ID 进行连接。"
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}
        try:
            id1_upper = comp1_id.strip().upper() # Ensure IDs are processed for consistency
            id2_upper = comp2_id.strip().upper()
            success, message = self.memory_manager.circuit_diagram.add_connection(id1_upper, id2_upper)
            if success:
                if "已存在连接" in message:
                     logger.info(f"[Action: ConnectComponents] {message}")
                     self.memory_manager.add_to_long_term(f"尝试连接已存在的连接: {id1_upper} <--> {id2_upper}")
                     return {"status": "success", "message": f"注意: {message}"}
                else:
                    logger.info(f"[Action: ConnectComponents] {message}")
                    self.memory_manager.add_to_long_term(f"连接了元件: {id1_upper} <--> {id2_upper}")
                    return {"status": "success", "message": f"操作成功: {message}"}
            else:
                logger.error(f"[Action: ConnectComponents] 连接失败: {message}")
                return {"status": "failure", "message": f"错误: {message}", "error": {"type": "ConnectionFailed", "details": message}}
        except Exception as e:
            msg=f"连接元件时发生意外的内部错误: {e}"
            logger.error(f"[Action: ConnectComponents] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 连接元件时发生未知内部错误。", "error": {"type": "UnexpectedError", "details": str(e)}}
    
    @register_tool(
        description="修改电路中一个已存在元件的值。",
        parameters={
            "type": "object", "properties": {
                "component_id": {"type": "string", "description": "要修改值元件的 ID。"},
                "new_value": {"type": "string", "description": "元件的新值。传空字符串或null可清除值。"} # Clarified new_value
            }, "required": ["component_id", "new_value"]
        }
    )
    def modify_component_value_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: ModifyComponentValue] 执行修改元件值操作。")
        component_id = arguments.get("component_id")
        new_value = arguments.get("new_value") 
        if not component_id or not isinstance(component_id, str) or not component_id.strip():
            msg = "必须提供有效的元件 ID。"
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": "component_id is required."}}
        try:
            cid_upper = component_id.strip().upper()
            component = self.memory_manager.circuit_diagram.get_component(cid_upper)
            if not component:
                msg = f"元件 '{cid_upper}' 不存在，无法修改其值。"
                logger.error(f"[Action: ModifyComponentValue] {msg}")
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": msg}}
            old_value_str = component.value if component.value is not None else "空"
            success = self.memory_manager.circuit_diagram.modify_component_value(cid_upper, new_value)
            if success:
                new_value_str = component.value if component.value is not None else "空" # Get updated value
                message = f"操作成功: 元件 '{cid_upper}' 的值已从 '{old_value_str}' 修改为 '{new_value_str}'。"
                logger.info(f"[Action: ModifyComponentValue] {message}")
                self.memory_manager.add_to_long_term(f"修改了元件 '{cid_upper}' 的值: 从 {old_value_str} 到 {new_value_str}")
                return {"status": "success", "message": message}
            else:
                msg = f"修改元件 '{cid_upper}' 的值失败。" # Should be caught above
                logger.error(f"[Action: ModifyComponentValue] {msg}")
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ModificationFailed", "details": "Failed to modify component value in diagram."}}
        except Exception as e:
            msg=f"修改元件值时发生意外的内部错误: {e}"
            logger.error(f"[Action: ModifyComponentValue] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 修改元件值时发生未知内部错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="从电路中移除一个指定的元件及其所有连接。",
        parameters={
            "type": "object", "properties": {
                "component_id": {"type": "string", "description": "要移除的元件的 ID。"}
            }, "required": ["component_id"]
        }
    )
    def remove_component_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: RemoveComponent] 执行移除元件操作。")
        component_id = arguments.get("component_id")
        if not component_id or not isinstance(component_id, str) or not component_id.strip():
            msg = "必须提供有效的元件 ID 进行移除。"
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": "component_id is required."}}
        try:
            cid_upper = component_id.strip().upper()
            component_to_remove = self.memory_manager.circuit_diagram.get_component(cid_upper)
            if not component_to_remove:
                msg = f"元件 '{cid_upper}' 不存在，无法移除。"
                logger.warning(f"[Action: RemoveComponent] {msg}")
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": msg}}
            component_str_for_log = str(component_to_remove)
            success = self.memory_manager.circuit_diagram.remove_component(cid_upper)
            if success:
                message = f"操作成功: 已移除元件 {component_str_for_log} 及其所有连接。"
                logger.info(f"[Action: RemoveComponent] {message}")
                self.memory_manager.add_to_long_term(f"移除了元件: {component_str_for_log}")
                return {"status": "success", "message": message}
            else: # Should be caught by not found
                msg = f"移除元件 '{cid_upper}' 失败。"
                logger.error(f"[Action: RemoveComponent] {msg}")
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "RemovalFailed", "details": "Failed to remove component from diagram."}}
        except Exception as e:
            msg=f"移除元件时发生意外的内部错误: {e}"
            logger.error(f"[Action: RemoveComponent] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 移除元件时发生未知内部错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="移除电路中两个指定元件之间的连接。",
        parameters={
            "type": "object", "properties": {
                "comp1_id": {"type": "string", "description": "第一个元件的 ID。"},
                "comp2_id": {"type": "string", "description": "第二个元件的 ID。"}
            }, "required": ["comp1_id", "comp2_id"]
        }
    )
    def remove_connection_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: RemoveConnection] 执行移除连接操作。")
        comp1_id = arguments.get("comp1_id")
        comp2_id = arguments.get("comp2_id")
        if not comp1_id or not isinstance(comp1_id, str) or not comp1_id.strip() or \
           not comp2_id or not isinstance(comp2_id, str) or not comp2_id.strip():
            msg = "必须提供两个有效的元件 ID 来移除连接。"
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}
        try:
            id1_upper = comp1_id.strip().upper()
            id2_upper = comp2_id.strip().upper()
            success, message = self.memory_manager.circuit_diagram.remove_connection(id1_upper, id2_upper)
            if success:
                logger.info(f"[Action: RemoveConnection] {message}")
                self.memory_manager.add_to_long_term(f"移除了连接: {id1_upper} <--> {id2_upper}")
                return {"status": "success", "message": f"操作成功: {message}"}
            else:
                logger.error(f"[Action: RemoveConnection] 移除连接失败: {message}")
                return {"status": "failure", "message": f"错误: {message}", "error": {"type": "RemoveConnectionFailed", "details": message}}
        except Exception as e:
            msg=f"移除连接时发生意外的内部错误: {e}"
            logger.error(f"[Action: RemoveConnection] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 移除连接时发生未知内部错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="获取当前电路的详细描述，包括所有已添加的元件及其值和所有连接。此为面向用户的视图。",
        parameters={"type": "object", "properties": {}}
    )
    def view_circuit_diagram_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: ViewCircuitDiagram] 执行查看电路图操作。")
        try:
            description = self.memory_manager.get_circuit_state_description()
            logger.info("[Action: ViewCircuitDiagram] 成功生成电路图描述。")
            return {"status": "success", "message": "已成功获取当前电路图的描述。", "data": {"description": description}}
        except Exception as e:
            msg=f"生成电路图描述时发生意外的内部错误: {e}"
            logger.error(f"[Action: ViewCircuitDiagram] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 获取电路图描述时发生未知错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="获取当前电路状态的内部摘要描述，供Agent内部决策使用。",
        parameters={"type": "object", "properties": {}} 
    )
    def describe_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: DescribeCircuit] 执行描述电路操作 (内部摘要)。")
        try:
            description = self.memory_manager.get_circuit_state_description()
            logger.info("[Action: DescribeCircuit] 成功生成电路内部摘要。")
            return {"status": "success", "message": "已成功获取当前电路的内部摘要描述。", "data": {"description": description}}
        except Exception as e:
            msg=f"生成电路内部摘要时发生意外的内部错误: {e}"
            logger.error(f"[Action: DescribeCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 获取电路内部摘要时发生未知错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    @register_tool(
        description="彻底清空当前的电路设计，移除所有元件和连接，并重置ID计数器。",
        parameters={"type": "object", "properties": {}}
    )
    def clear_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        logger.info("[Action: ClearCircuit] 执行清空电路操作。")
        try:
            self.memory_manager.clear_circuit()
            logger.info("[Action: ClearCircuit] 电路状态已成功清空。")
            self.memory_manager.add_to_long_term("执行了清空电路操作。")
            success_message = "操作成功: 当前电路已彻底清空。"
            return {"status": "success", "message": success_message}
        except Exception as e:
            msg=f"清空电路时发生意外的内部错误: {e}"
            logger.error(f"[Action: ClearCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 清空电路时发生未知错误。", "error": {"type": "UnexpectedError", "details": str(e)}}

    # --- Orchestration Layer Method (with Interaction Retry) ---
    async def process_user_request(self, user_request: str) -> str:
        """
        处理用户请求的顶层方法，包含整个交互流程的重试机制。
        """
        overall_request_start_time = time.monotonic()
        logger.info(f"\n{'='*25} 开始处理用户请求 (顶层) {'='*25}")
        logger.info(f"[Orchestrator_TopLevel] 收到用户指令: \"{user_request}\"")

        if not user_request or user_request.isspace():
            logger.info("[Orchestrator_TopLevel] 用户指令为空或仅包含空白。")
            await async_print("\n您的指令似乎是空的，请重新输入！")
            return "<think>用户输入为空或空白，无需处理。</think>\n\n请输入您的指令！"

        # 记录在本次用户请求进入前的短期记忆状态，用于交互重试时的恢复
        # 我们只关心对话历史，system prompt通常在最开始，或者由_attempt_process_user_request_once动态生成
        # 为了简化，我们深拷贝整个 short_term，然后在 _attempt_process_user_request_once 开始时再精确构造
        initial_short_term_memory_before_this_request = copy.deepcopy(self.memory_manager.short_term)
        
        final_report_from_attempt = ""
        last_error_message_for_interaction = "未能成功处理请求。" # Default error for overall failure

        for interaction_attempt in range(self.max_interaction_attempts):
            current_interaction_attempt = interaction_attempt + 1
            logger.info(f"[Orchestrator_TopLevel] === 开始第 {current_interaction_attempt}/{self.max_interaction_attempts} 次交互尝试 ===")
            if current_interaction_attempt > 1:
                await async_print(f"--- 处理您的请求时遇到一些挑战，正在进行第 {current_interaction_attempt}/{self.max_interaction_attempts} 次尝试... ---")
                # 恢复短期记忆到本次用户请求处理开始前的状态
                # 然后再添加当前用户请求，确保每次交互尝试都从一个干净的用户请求开始
                self.memory_manager.short_term = copy.deepcopy(initial_short_term_memory_before_this_request)
                logger.info(f"[Orchestrator_TopLevel] (重试) 短期记忆已恢复到处理此用户请求前的状态 (数量: {len(self.memory_manager.short_term)})。")
            
            # 将当前用户请求添加到（可能已恢复的）短期记忆中
            # 这一步确保即便是重试，用户请求也是当前上下文的一部分
            try:
                # 检查用户消息是否已在 initial_short_term_memory_before_this_request 的最后一条（避免重复添加）
                # 简单起见，这里直接添加，add_to_short_term 里的修剪机制会处理长度
                # 更精细的控制是确保 user_request 只被添加一次到整个会话历史。
                # 鉴于我们恢复的是 *此请求之前* 的历史，再添加当前请求是正确的。
                self.memory_manager.add_to_short_term({"role": "user", "content": user_request})
                if current_interaction_attempt == 1: # 只在第一次尝试时记录这个，避免重试时重复日志
                    logger.info(f"[Orchestrator_TopLevel] 用户指令 '{user_request[:50]}...' 已记录并添加到短期记忆。")
            except Exception as e: # Should be rare for add_to_short_term
                logger.error(f"[Orchestrator_TopLevel] (尝试 {current_interaction_attempt}) 添加用户消息到短期记忆时出错: {e}", exc_info=True)
                await async_print(f"\n🔴 抱歉，我在记录您的指令时遇到了内部问题 ({e})！(尝试 {current_interaction_attempt})")
                # 此类错误通常是内部问题，可能不适合直接重试整个交互，但我们还是继续，看能否恢复
                last_error_message_for_interaction = f"内部记忆错误: {e}"
                final_report_from_attempt = f"<think>添加用户消息到短期记忆失败: {e}</think>\n\n抱歉，我在处理您的指令时遇到了内部记忆错误，请稍后再试。"
                continue # 继续下一次交互尝试（如果有）

            # 调用实际处理单次交互的内部方法
            success, result_report, error_details = await self._attempt_process_user_request_once(user_request)
            final_report_from_attempt = result_report # Store the report from this attempt
            last_error_message_for_interaction = error_details if error_details else "未知交互错误。"

            if success:
                logger.info(f"[Orchestrator_TopLevel] === 第 {current_interaction_attempt}/{self.max_interaction_attempts} 次交互尝试成功 ===")
                request_total_duration = time.monotonic() - overall_request_start_time
                logger.info(f"\n{'='*25} 用户请求处理完毕 (顶层成功, 总耗时: {request_total_duration:.3f} 秒) {'='*25}\n")
                return final_report_from_attempt # 成功，返回结果
            else:
                logger.warning(f"[Orchestrator_TopLevel] === 第 {current_interaction_attempt}/{self.max_interaction_attempts} 次交互尝试失败。失败原因: {error_details} ===")
                # 如果是最后一次交互尝试仍然失败，则准备退出循环并返回最后一次的失败报告
                if current_interaction_attempt == self.max_interaction_attempts:
                    logger.error(f"[Orchestrator_TopLevel] 所有 {self.max_interaction_attempts} 次交互尝试均失败。最后一次错误: {error_details}")
                    await async_print(f"\n🔴 非常抱歉，我多次尝试处理您的指令（{self.max_interaction_attempts}次）都未能成功。最后遇到的问题是：{error_details}")
                    # final_report_from_attempt 已经包含了最后一次尝试的详细报告(可能含<think>)
                    break # 跳出交互重试循环
                else:
                    # 还有交互重试次数，将进行下一次尝试
                    logger.info(f"[Orchestrator_TopLevel] 将进行下一次交互尝试...")
                    await asyncio.sleep(1.0 + interaction_attempt) # 简单的线性增加延迟，避免立即重试轰炸

        # 如果所有交互尝试都失败了
        request_total_duration = time.monotonic() - overall_request_start_time
        logger.error(f"[Orchestrator_TopLevel] 最终未能成功处理用户请求 \"{user_request[:50]}...\" 经过 {self.max_interaction_attempts} 次尝试。总耗时: {request_total_duration:.3f} 秒。")
        # 返回最后一次尝试的报告，它应该已经包含了错误信息和可能的<think>
        return final_report_from_attempt if final_report_from_attempt else f"<think>所有交互尝试均失败。最后错误: {last_error_message_for_interaction}</think>\n\n非常抱歉，系统多次尝试处理您的请求均未成功，请稍后再试或联系技术支持。"

    async def _attempt_process_user_request_once(self, user_request_for_this_attempt: str) -> Tuple[bool, str, Optional[str]]:
        """
        执行一次完整的用户请求处理流程（感知->规划->行动->观察->响应）。
        返回: (是否成功, 最终报告字符串, 错误详情（如果失败）)
        注意: 此方法期望 self.memory_manager.short_term 已经包含了当前用户请求作为最后一条消息。
        """
        attempt_start_time = time.monotonic()
        logger.info("--- [阶段 1 - 单次尝试] 感知与记忆已在顶层处理 ---")
        # 用户请求已由 process_user_request 添加到短期记忆

        # --- 阶段 2: 规划 (第一次 LLM 调用 - 自定义 JSON 模式，带重试) ---
        logger.info("\n--- [阶段 2 - 单次尝试] 规划 (请求 LLM 生成执行计划) ---")
        await async_print("--- 正在请求智能大脑分析指令并生成执行计划 (JSON)... ---")
        memory_context = self.memory_manager.get_memory_context_for_prompt()
        tool_schemas_for_prompt = self._get_tool_schemas_for_prompt()
        if not self.tools_registry: logger.warning("[Orchestrator_Attempt] 没有可用的工具。")
        system_prompt_planning = self._get_planning_prompt(tool_schemas_for_prompt, memory_context)
        
        # 构建发送给LLM的消息列表：System Prompt + (短期记忆中除了System Prompt之外的所有内容)
        # 确保短期记忆中的第一条是最新的System Prompt，其余的是对话历史。
        messages_for_llm1 = [{"role": "system", "content": system_prompt_planning}]
        # 从恢复的短期记忆中提取非系统消息 (这些消息是此用户请求之前的历史 + 当前用户请求)
        messages_for_llm1.extend([msg for msg in self.memory_manager.short_term if msg.get("role") != "system"])

        first_llm_response = None; thinking_process = "未能提取思考过程。"; plan_dict = None
        parser_error_msg = ""; raw_content_from_llm = None
        planning_attempt = 0; max_planning_attempts = 1 + self.planning_llm_retries

        while planning_attempt < max_planning_attempts:
            planning_attempt += 1
            logger.info(f"[Orchestrator_Attempt] 尝试第 {planning_attempt}/{max_planning_attempts} 次调用规划 LLM...")
            await async_print(f"    (与大脑规划沟通尝试 {planning_attempt}/{max_planning_attempts})...")
            try:
                first_llm_response = await self.llm_interface.call_llm(messages=messages_for_llm1, use_tools=False)
                logger.info(f"[Orchestrator_Attempt] 第 {planning_attempt} 次规划 LLM 调用完成。")
                logger.info(f"--- [阶段 3 - 单次尝试 - 规划尝试 {planning_attempt}] 解析 LLM 的规划响应 ---")
                if not first_llm_response or not first_llm_response.choices:
                     logger.error(f"[Orchestrator_Attempt] 第 {planning_attempt} 次规划 LLM 响应无效或缺少 'choices'。")
                     parser_error_msg = "LLM 响应无效或缺少 'choices'。"
                     # 对于这种LLM级别的问题，如果多次尝试都这样，可能需要触发整个交互的重试
                     if planning_attempt == max_planning_attempts: # All planning retries exhausted
                        return False, f"<think>规划LLM响应在所有 {max_planning_attempts} 次尝试后均无效。最后思考: {thinking_process}</think>\n\n抱歉，智能大脑未能提供有效的规划信息。", "规划LLM响应无效"
                     continue # Try next planning attempt

                response_message = first_llm_response.choices[0].message
                raw_content_from_llm = getattr(response_message, 'content', None)
                logger.debug(f"[Orchestrator_Attempt] 第 {planning_attempt} 次规划 LLM 原始 Content (前 1000 字): >>>\n{str(raw_content_from_llm)[:1000]}...\n<<<")
                if getattr(response_message, 'tool_calls', None):
                    logger.warning(f"[Orchestrator_Attempt] 警告 (规划尝试 {planning_attempt})：LLM 在自定义 JSON 模式下意外返回了 'tool_calls' 字段！")
                
                thinking_process, plan_dict, parser_error_msg = self.output_parser.parse_planning_response(response_message)
                if plan_dict is not None and not parser_error_msg:
                    logger.info(f"[Orchestrator_Attempt] 第 {planning_attempt} 次尝试成功解析并验证自定义 JSON 计划！")
                    break # 规划成功，跳出规划重试循环
                else:
                    logger.warning(f"[Orchestrator_Attempt] 第 {planning_attempt} 次尝试解析 JSON 失败: {parser_error_msg}")
                    # 如果所有规划重试都失败了
                    if planning_attempt == max_planning_attempts:
                        raw_content_snippet = f"\n最后一次 LLM 原始响应片段:\n{str(raw_content_from_llm)[:300]}..." if raw_content_from_llm else ""
                        err_detail = f"解析 LLM 返回的自定义 JSON 失败 (所有 {max_planning_attempts} 次规划尝试均失败)。最终错误: {parser_error_msg}{raw_content_snippet}"
                        return False, f"<think>{thinking_process}\n{err_detail}</think>\n\n抱歉！我多次尝试解析智能大脑生成的执行计划均失败了。", err_detail
            except ConnectionError as conn_err:
                logger.error(f"[Orchestrator_Attempt] 第 {planning_attempt} 次规划 LLM 调用失败 (连接错误): {conn_err}", exc_info=True)
                parser_error_msg = f"LLM 调用连接错误: {conn_err}"
                if planning_attempt == max_planning_attempts:
                    return False, f"<think>尝试 {max_planning_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！我多次尝试联系智能大脑进行规划都失败了。", f"规划LLM连接错误: {conn_err}"
            except Exception as e: # Other unexpected errors during planning LLM call/parsing
                logger.error(f"[Orchestrator_Attempt] 第 {planning_attempt} 次规划 LLM 调用或解析过程中发生严重错误: {e}", exc_info=True)
                parser_error_msg = f"LLM 调用或响应解析时发生错误: {e}"
                if planning_attempt == max_planning_attempts:
                    return False, f"<think>尝试 {max_planning_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！在规划阶段遇到了严重的内部错误 ({e})。", f"规划LLM/解析时意外错误: {e}"
        
        # 此时，如果 plan_dict 仍然为 None，说明所有规划尝试都失败了，已在循环内返回 (False, ...)
        # 因此，如果代码执行到这里，plan_dict 应该是非 None 的
        if plan_dict is None: # Defensive check, should not be reached if logic above is correct
            logger.error("[Orchestrator_Attempt] 内部逻辑错误：规划重试循环后 plan_dict 仍为 None。")
            return False, "<think>内部逻辑错误，未能获得规划。</think>\n\n抱歉，系统出现内部逻辑错误，无法处理您的请求。", "内部规划逻辑错误"

        logger.info("[Orchestrator_Attempt] 成功获取并验证自定义 JSON 计划。")
        logger.debug(f"[Orchestrator_Attempt] 解析出的计划详情: {json.dumps(plan_dict, indent=2, ensure_ascii=False)}")
        await async_print("--- 大脑已生成执行计划 ---")

        try: # 将LLM的规划响应（原始消息对象）添加到短期记忆
             if first_llm_response and first_llm_response.choices:
                 assistant_plan_message = first_llm_response.choices[0].message
                 assistant_raw_response_for_memory = assistant_plan_message.model_dump(exclude_unset=True)
                 self.memory_manager.add_to_short_term(assistant_raw_response_for_memory)
                 logger.debug("[Orchestrator_Attempt] LLM 的原始规划响应已添加至短期记忆。")
             else: logger.error("[Orchestrator_Attempt] 规划成功但无法获取有效的 LLM 响应对象以存入记忆。")
        except Exception as mem_err: # 添加记忆失败不应阻止流程，但要记录
             logger.error(f"[Orchestrator_Attempt] 添加 LLM 规划响应到短期记忆失败: {mem_err}", exc_info=True)

        is_tool_calls = plan_dict.get("is_tool_calls", False)
        tool_list_from_plan = plan_dict.get("tool_list")
        direct_reply_from_plan = plan_dict.get("direct_reply")

        if is_tool_calls:
            logger.info("[Orchestrator_Attempt] 决策：根据 JSON 计划执行工具。")
            if not isinstance(tool_list_from_plan, list) or not tool_list_from_plan:
                 err_detail = "规划错误: 'is_tool_calls' 为 true 但 'tool_list' 不是有效的非空列表！"
                 logger.error(f"[Orchestrator_Attempt] {err_detail}")
                 await async_print("\n🔴 内部错误：执行计划要求调用工具，但工具列表无效。")
                 return False, f"<think>{thinking_process}\n{err_detail} Plan: {plan_dict}</think>\n\n抱歉，执行计划似乎存在内部问题。", err_detail
            try: # 排序工具列表
                sorted_tool_list = sorted(tool_list_from_plan, key=lambda x: x.get('index', float('inf')))
                # Optional: check continuity of indices (already in parser)
                logger.info(f"[Orchestrator_Attempt] 工具列表已按 index 排序，共 {len(sorted_tool_list)} 个工具需要执行。")
            except Exception as sort_err:
                 err_detail = f"排序工具列表时出错: {sort_err}"
                 logger.error(f"[Orchestrator_Attempt] {err_detail}", exc_info=True)
                 await async_print("\n🔴 抱歉，我在安排工具执行顺序时遇到了内部问题。")
                 return False, f"<think>{thinking_process}\n{err_detail}</think>\n\n抱歉，我在处理工具执行顺序时遇到了内部错误。", err_detail

            # 将自定义工具列表转换为模拟ToolCall列表
            mock_tool_calls_for_executor = []
            conversion_successful = True
            for tool_item in sorted_tool_list:
                tool_name = tool_item.get("toolname"); params_dict = tool_item.get("params", {}); index = tool_item.get("index")
                params_hash = hash(json.dumps(params_dict, sort_keys=True)) & 0xffff
                mock_id = f"call_{index}_{tool_name[:8]}_{params_hash:x}"
                try: params_str = json.dumps(params_dict)
                except TypeError as json_dump_err:
                    logger.error(f"转换工具 {tool_name} (index {index}) 的参数字典为 JSON 字符串失败: {json_dump_err}. Params: {params_dict}", exc_info=True)
                    conversion_successful = False; params_str = "{}"
                mock_tool_calls_for_executor.append({"id": mock_id, "type": "function", "function": {"name": tool_name, "arguments": params_str}})
            if not conversion_successful: logger.warning("[Orchestrator_Attempt] 注意: 转换自定义工具列表时遇到参数序列化问题。")
            logger.info(f"[Orchestrator_Attempt] 成功将自定义工具列表转换为 {len(mock_tool_calls_for_executor)} 个模拟 ToolCall 对象。")
            
            # --- 阶段 4: 行动执行 (调用 ToolExecutor - 异步 & 失败中止 & 工具级重试) ---
            logger.info("\n--- [阶段 4 - 单次尝试] 行动 (执行工具) ---")
            num_tools_to_run = len(mock_tool_calls_for_executor)
            await async_print(f"--- 正在按计划执行 {num_tools_to_run} 个操作 (带工具级重试, 若有失败则中止)... ---")
            tool_execution_results = []; num_actually_executed = 0
            try:
                tool_execution_results = await self.tool_executor.execute_tool_calls(mock_tool_calls_for_executor)
                num_actually_executed = len(tool_execution_results)
                logger.info(f"[Orchestrator_Attempt] ToolExecutor 完成了 {num_actually_executed}/{num_tools_to_run} 个工具执行尝试。")
                if num_actually_executed < num_tools_to_run:
                     logger.warning(f"[Orchestrator_Attempt] 由于中途有工具失败，计划中的后续 {num_tools_to_run - num_actually_executed} 个工具未执行。")
                await async_print(f"--- {num_actually_executed}/{num_tools_to_run} 个操作已执行 ---")
            except Exception as e: # ToolExecutor 顶层错误
                 err_detail = f"ToolExecutor 执行过程中发生顶层意外错误: {e}"
                 logger.error(f"[Orchestrator_Attempt] {err_detail}", exc_info=True)
                 await async_print(f"\n🔴 抱歉，执行工具时系统发生严重错误 ({e})！")
                 # 此类错误可能表示ToolExecutor本身有问题，可能值得进行整个交互的重试
                 return False, f"<think>{thinking_process}\n{err_detail}</think>\n\n抱歉，系统在执行工具时发生严重内部错误。", err_detail
            
            # --- 阶段 5: 观察与记忆更新 (工具结果) ---
            logger.info("\n--- [阶段 5 - 单次尝试] 观察 (处理工具结果并更新记忆) ---")
            num_tool_results_added = 0
            if not tool_execution_results: logger.warning("[Orchestrator_Attempt] ToolExecutor 未返回任何执行结果。")
            else:
                for exec_result in tool_execution_results:
                    tool_call_id_for_memory = exec_result.get('tool_call_id', 'unknown_mock_id')
                    result_dict = exec_result.get('result', {"status": "unknown", "message": "执行结果丢失"})
                    if not isinstance(result_dict, dict):
                        logger.warning(f"工具 {tool_call_id_for_memory} 的结果不是字典格式: {result_dict}")
                        result_dict = {"status": "unknown", "message": "非字典格式的工具结果", "raw_result": str(result_dict)}
                    try: result_content_str = json.dumps(result_dict, indent=2, ensure_ascii=False, default=str)
                    except Exception as json_dump_error:
                        logger.error(f"序列化工具 {tool_call_id_for_memory} 的结果字典失败: {json_dump_error}. Result: {result_dict}")
                        result_content_str = f'{{"status": "serialization_error", "message": "Failed to serialize result dict: {json_dump_error}", "original_result_repr": "{repr(result_dict)[:100]}..."}}'
                    tool_message = {"role": "tool", "tool_call_id": tool_call_id_for_memory, "content": result_content_str}
                    try: self.memory_manager.add_to_short_term(tool_message); num_tool_results_added += 1
                    except Exception as mem_err: logger.error(f"添加工具 {tool_call_id_for_memory} 结果到短期记忆失败: {mem_err}", exc_info=True)
            logger.info(f"[Orchestrator_Attempt] {num_tool_results_added}/{len(tool_execution_results)} 个工具执行结果已添加至短期记忆。")
            logger.debug("工具执行后的电路状态:\n%s", self.memory_manager.get_circuit_state_description())

            # --- 阶段 6: 回复生成 (第二次 LLM 调用 - 异步) ---
            logger.info("\n--- [阶段 6 - 单次尝试] 响应生成 (请求 LLM 总结结果) ---")
            await async_print("--- 正在请求智能大脑总结操作结果并生成最终报告... ---")
            memory_context_final = self.memory_manager.get_memory_context_for_prompt()
            tool_schemas_final = self._get_tool_schemas_for_prompt()
            tools_were_skipped = num_actually_executed < num_tools_to_run
            system_prompt_response_generation = self._get_response_generation_prompt(memory_context_final, tool_schemas_final, tools_were_skipped)
            messages_for_llm2 = [{"role": "system", "content": system_prompt_response_generation}] + self.memory_manager.short_term
            
            second_llm_response = None; final_report = ""
            try:
                 logger.info("[Orchestrator_Attempt] 准备执行第二次 LLM 调用 (用于生成最终响应)...")
                 second_llm_response = await self.llm_interface.call_llm(messages=messages_for_llm2, use_tools=False)
                 logger.info("[Orchestrator_Attempt] 第二次 LLM 调用完成。")
                 await async_print("--- 大脑已生成最终报告 ---")
                 logger.info("\n--- [阶段 7 - 单次尝试] 解析最终报告 ---")
                 if not second_llm_response or not second_llm_response.choices or not second_llm_response.choices[0].message or not second_llm_response.choices[0].message.content:
                     err_detail = "第二次 LLM 响应无效或内容为空。"
                     logger.error(f"[Orchestrator_Attempt] {err_detail}")
                     final_thinking = thinking_process + f"\n错误: {err_detail}"
                     final_reply = "抱歉，我在总结结果时遇到了问题，智能大脑没有返回有效的报告内容。"
                     # 此处失败可能也需要触发顶层交互重试
                     return False, f"<think>{final_thinking}</think>\n\n{final_reply}", err_detail
                 else: # LLM响应看似有效，进行解析
                     final_response_message = second_llm_response.choices[0].message
                     raw_final_content = final_response_message.content
                     logger.debug(f"[Orchestrator_Attempt] 第二次 LLM 原始 Content (前 1000 字): >>>\n{str(raw_final_content)[:1000]}...\n<<<")
                     final_thinking, final_reply = self.output_parser._parse_llm_text_content(raw_final_content)
                     # 检查解析后的 final_reply 是否仍然包含意外的 JSON (如 OutputParser 中增强的检测)
                     if "格式不正确（包含了意外的JSON计划）" in final_reply or \
                        (final_reply.strip().startswith("{") and ("is_tool_calls" in final_reply or "tool_list" in final_reply)):
                         err_detail = "最终LLM报告格式错误：意外地包含了JSON计划结构。"
                         logger.error(f"[Orchestrator_Attempt] {err_detail}. Raw final content was: {raw_final_content[:300]}...")
                         # 使用原始的 thinking_process 可能更相关
                         return False, f"<think>{thinking_process}\n错误: {err_detail}</think>\n\n抱歉，智能大脑生成的最终报告格式不正确，无法完整显示。", err_detail

                     try: # 添加最终回复到短期记忆
                         final_assistant_message_dump = final_response_message.model_dump(exclude_unset=True)
                         self.memory_manager.add_to_short_term(final_assistant_message_dump)
                         logger.debug("[Orchestrator_Attempt] 最终 LLM 回复已添加至短期记忆。")
                     except Exception as mem_err:
                         logger.error(f"添加最终回复到短期记忆失败: {mem_err}", exc_info=True) # Non-fatal for this attempt
                 final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()
            except ConnectionError as conn_err_resp_gen: # LLM调用连接错误
                err_detail = f"第二次LLM调用（响应生成）连接失败: {conn_err_resp_gen}"
                logger.critical(f"[Orchestrator_Attempt] {err_detail}", exc_info=True)
                return False, f"<think>{thinking_process}\n错误: {err_detail}</think>\n\n抱歉，在为您准备最终报告时与智能大脑的连接中断。", err_detail
            except Exception as e_resp_gen: # 其他响应生成阶段错误
                 err_detail = f"第二次 LLM 调用或最终报告处理失败: {e_resp_gen}"
                 logger.critical(f"[Orchestrator_Attempt] {err_detail}", exc_info=True)
                 final_thinking_err = thinking_process + f"\n第二次 LLM 调用或最终报告处理失败: {e_resp_gen}"
                 final_reply_err = f"抱歉，在为您准备最终报告时遇到了严重的内部错误 ({e_resp_gen})！"
                 final_report = f"<think>{final_thinking_err}</think>\n\n{final_reply_err}".rstrip()
                 return False, final_report, err_detail # 标记为失败，并返回包含错误的报告
            
            attempt_duration = time.monotonic() - attempt_start_time
            logger.info(f"\n{'='*25} 单次交互尝试完毕 (工具调用路径, 耗时: {attempt_duration:.3f} 秒) {'='*25}\n")
            return True, final_report, None # 成功
        else: # is_tool_calls is False (直接回复路径)
            logger.info("[Orchestrator_Attempt] 决策：根据 JSON 计划直接回复，不执行工具。")
            await async_print("--- 大脑认为无需执行操作，将直接回复... ---")
            if direct_reply_from_plan and isinstance(direct_reply_from_plan, str) and direct_reply_from_plan.strip():
                logger.info("[Orchestrator_Attempt] 使用计划中提供的 'direct_reply' 作为最终回复。")
                final_thinking = thinking_process
                final_reply = direct_reply_from_plan
            else: # 规划错误：is_tool_calls false 但 direct_reply 无效
                err_detail = "规划错误: 'is_tool_calls' 为 false 但 'direct_reply' 无效或缺失！"
                logger.error(f"[Orchestrator_Attempt] {err_detail}")
                await async_print("\n🔴 内部错误：计划无需操作，但大脑没有提供回复内容！")
                final_thinking = thinking_process + f"\n{err_detail}"
                final_reply = "我理解现在不需要执行操作，但是智能大脑没有提供相应的回复。这可能是一个规划错误。"
                return False, f"<think>{final_thinking}</think>\n\n{final_reply}", err_detail
            final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()
            # 第一次LLM的规划响应已加入短期记忆，其中包含了这个direct_reply
            attempt_duration = time.monotonic() - attempt_start_time
            logger.info(f"\n{'='*25} 单次交互尝试完毕 (直接回复路径, 耗时: {attempt_duration:.3f} 秒) {'='*25}\n")
            return True, final_report, None # 成功

    # --- Helper Methods for Prompts ---
    def _get_tool_schemas_for_prompt(self) -> str:
        if not self.tools_registry: return "  (无可用工具)"
        tool_schemas = []
        for name, schema in self.tools_registry.items():
            desc = schema.get('description', '无描述。')
            params = schema.get('parameters', {})
            props = params.get('properties', {})
            req = params.get('required', [])
            param_desc_parts = []
            if props:
                for k, v in props.items():
                    p_type = v.get('type', 'any')
                    p_desc = v.get('description', '')
                    p_req = '(必须)' if k in req else '(可选)'
                    param_desc_parts.append(f"{k}: {p_type} {p_req} '{p_desc}'")
                param_desc_str = "; ".join(param_desc_parts)
            else: param_desc_str = "无参数"
            tool_schemas.append(f"  - `{name}`: {desc} (参数: {param_desc_str})")
        return "\n".join(tool_schemas)

    def _get_planning_prompt(self, tool_schemas_desc: str, memory_context: str) -> str:
        # (与之前版本prompt内容一致，此处为简洁省略，实际代码中应保留完整prompt)
        return (
            "你是一位顶尖的、极其严谨的电路设计编程助理。你的行为必须专业、精确，并严格遵循指令。\n"
            "你的任务是：分析用户的最新指令、对话历史以及当前的电路状态，然后**严格按照下面描述的固定格式**生成一个包含执行计划的 JSON 对象。\n"
            "**绝对禁止**使用任何形式的 Function Calling 或生成 `tool_calls` 字段。你的**唯一输出**必须由两部分组成：\n"
            "1.  一个 `<think>...</think>` XML 块：在其中详细阐述你的思考过程。\n"
            "2.  **紧随其后**，**必须**是一个**单一的、格式完全正确的 JSON 对象**。**不允许在 JSON 对象的前面或后面添加任何额外的文字！**\n\n"
            "**JSON 对象格式规范 (必须严格遵守):**\n"
            "  - `is_tool_calls` (boolean): **必须**。如果需要执行工具，则为 `true`。如果不需要，则为 `false`。\n"
            "  - `tool_list` (array<object> | null): **必须**。\n"
            "     - 当 `is_tool_calls` 为 `true` 时: **必须**是一个包含**一个或多个**工具调用对象的数组，按期望执行顺序列出。\n"
            "     - 当 `is_tool_calls` 为 `false` 时: 此字段**必须**是 `null` 或 `[]`。\n"
            "     每个工具调用对象**必须**包含: `toolname` (string), `params` (object), `index` (integer, 从1开始连续)。\n"
            "  - `direct_reply` (string | null): **必须**。\n"
            "     - 当 `is_tool_calls` 为 `false` 时: **必须**包含直接回复用户的完整文本，不能为空。\n"
            "     - 当 `is_tool_calls` 为 `true` 时: 此字段**必须**是 `null`。\n\n"
            "**可用工具列表与参数规范:**\n"
            f"{tool_schemas_desc}\n\n"
            "**当前电路状态与记忆:**\n"
            f"{memory_context}\n\n"
            "**最后再次强调：你的回复格式必须严格是 `<think>思考过程</think>` 后面紧跟着一个符合上述规范的 JSON 对象。** JSON 的语法（括号、引号、逗号、数据类型）和结构（必需字段、条件字段）都必须完全正确，否则后续处理会失败。"
        )

    def _get_response_generation_prompt(self, memory_context: str, tool_schemas_desc: str, tools_were_skipped: bool) -> str:
        # (与之前版本prompt内容一致，此处为简洁省略，实际代码中应保留完整prompt)
        skipped_info = ""
        if tools_were_skipped:
            skipped_info = "\n**重要提示：** 由于先前有工具执行失败，部分计划步骤可能未完成。请在报告中说明此情况，并总结当前任务的最终状态。"
        return (
            "你是一位顶尖的电路设计编程助理，经验丰富，技术精湛，并且擅长清晰地汇报工作结果。\n"
            "你的当前任务是：基于到目前为止的完整对话历史（包括用户最初的指令、你之前的思考和规划、以及所有已执行工具的结果），生成最终的、面向用户的文本回复。\n"
            "**关键信息来源是角色为 'tool' 的消息**: 每条 'tool' 消息都对应一个之前执行的工具调用。其 `content` 字段是一个 JSON 字符串，包含了该工具执行的关键信息，特别是 `status` 字段（指示 'success' 或 'failure'）和 `message` 字段（描述结果或错误）。可能还包含 `error` 字段提供失败的详细技术信息。\n"
            "**你的报告必须：**\n"
            "1.  仔细阅读并理解所有历史消息，**特别是要解析每条 'tool' 消息中的 JSON 内容**，准确把握每个已执行工具的**最终状态 (`status`)** 和结果 (`message`)。\n"
            "2.  清晰地向用户总结所有**已执行**工具操作的结果。\n"
            f"{skipped_info}\n"
            "3.  综合以上信息，回答用户最初的问题或确认任务的完成情况。如果任务因工具失败而未能完全完成，请明确说明当前的状态。\n"
            "4.  **严格按照以下固定格式**生成你的回复：\n"
            "   a. **思考过程:** 首先，在 `<think>` 和 `</think>` 标签之间，详细阐述你的反思和报告组织思路。\n"
            "   b. **正式回复:** 在 `</think>` 标签之后，紧跟着面向用户的正式文本回复。\n"
            "**最终输出格式必须严格是:**\n"
            "`<think>你的详细思考过程</think>\n\n你的正式回复文本`\n"
            "(注意：`</think>` 标签后必须恰好是两个换行符 `\\n\\n`，然后直接是正式回复文本。)\n"
            "**重要：** 在这个阶段，你**绝对不能**再生成任何工具调用或 JSON 对象。你的唯一输出应该是包含 `<think>` 块和正式回复文本的字符串。**若在此阶段输出JSON，回复将被视为无效！**"
            "\n\n"
            "**上下文参考信息:**\n"
            "【当前电路状态与记忆】\n"
            f"{memory_context}\n"
            "【我的可用工具列表 (仅供你参考，不应再次调用)】\n"
            f"{tool_schemas_desc}\n"
        )

# --- 异步主函数 (应用程序入口) ---
async def main():
    await async_print("=" * 70)
    await async_print("🚀 启动 OpenManus 电路设计 Agent (全流程重试增强版) 🚀")
    await async_print(f"   日志文件保存在: {log_filename}")
    await async_print("=" * 70)
    logger.info("[Main] 开始 Agent 初始化...")

    api_key = os.environ.get("ZHIPUAI_API_KEY")
    if not api_key:
        logger.warning("[Main] 环境变量 ZHIPUAI_API_KEY 未设置。")
        await async_print("\n需要您的智谱AI API Key。")
        try: api_key = input("👉 请在此输入您的智谱AI API Key: ").strip()
        except (EOFError, KeyboardInterrupt):
            await async_print("\n输入被中断。程序退出。"); logger.info("[Main] 用户中断了 API Key 输入。"); return
        if not api_key:
            await async_print("\n错误：未提供 API Key。程序退出。"); logger.critical("[Main] 用户未提供 API Key。"); return
        logger.info("[Main] 已通过手动输入获取 API Key。")
    agent = None
    try:
        agent = CircuitDesignAgent(
            api_key=api_key,
            model_name="glm-4-flash", # Example, use appropriate model
            planning_llm_retries=1,    # Retries for planning LLM call
            max_short_term_items=25,   # Short-term memory capacity
            tool_executor_retries=3,   # Total attempts for each tool (1 initial + 2 retries)
            max_interaction_retries=1  # Retries for the whole interaction (1 initial + 1 retry = 2 total attempts)
        )
        await async_print("\n🎉 Agent 初始化成功！已准备就绪。")
        await async_print("\n您可以尝试以下指令:")
        await async_print("  - '给我加个1k电阻R1和3V电池B1'")
        await async_print("  - '连接R1和B1'")
        await async_print("  - '修改R1的值为2k'")
        await async_print("  - '移除B1和R1的连接'")
        await async_print("  - '移除元件R1'")
        await async_print("  - '查看电路图'")
        await async_print("  - '清空电路'")
        await async_print("  - '你好'")
        await async_print("  - '请你实现一个RC滤波电路，然后让它复杂一点，至少五个元件'") # Test multi-step that might fail
        await async_print("  - 输入 '退出' 来结束程序")
        await async_print("-" * 70)
    except Exception as e:
        logger.critical(f"[Main] Agent 初始化失败: {e}", exc_info=True)
        await async_print(f"\n🔴 Agent 初始化失败！错误: {e}。程序退出。"); return
    try:
        while True:
            try:
                user_input = ""
                try: user_input = input("用户 > ").strip()
                except (EOFError, KeyboardInterrupt): raise 
                if user_input.lower() in ['退出', 'quit', 'exit', '再见', '结束', 'bye']:
                    await async_print("\n收到退出指令。感谢您的使用！👋"); logger.info("[Main] 收到退出指令。"); break
                if not user_input: continue
                start_process_time = time.monotonic()
                response = await agent.process_user_request(user_input) # This now has top-level retry
                process_duration = time.monotonic() - start_process_time
                await async_print(f"\n📝 Agent 回复 (总耗时: {process_duration:.3f} 秒):")
                await async_print(response)
                await async_print("-" * 70)
            except KeyboardInterrupt: await async_print("\n用户操作被中断。"); logger.info("[Main] 用户中断当前请求。"); break
            except EOFError: await async_print("\n输入流意外结束。"); logger.info("[Main] 输入流结束 (EOF)。"); break
            except Exception as loop_err: # Catch errors within a single user request processing loop
                logger.error(f"[Main] 处理指令 '{user_input[:50]}...' 时发生意外错误: {loop_err}", exc_info=True)
                await async_print(f"\n🔴 Agent 运行时错误:")
                error_report = f"<think>处理指令 '{user_input[:50]}...' 时发生内部错误: {loop_err}\n{traceback.format_exc()}</think>\n\n抱歉，我在执行您的指令时遇到了意外问题 ({loop_err})！"
                await async_print(error_report)
                await async_print("-" * 70)
                continue # Continue to next user input
    except Exception as outer_loop_err: # Catch errors in the main while loop itself
        logger.critical(f"[Main] 主交互循环外发生未处理异常: {outer_loop_err}", exc_info=True)
        await async_print(f"\n🔴 严重系统错误导致交互循环终止: {outer_loop_err}。")
    finally:
        logger.info("[Main] 主交互循环结束。")
        await async_print("\n正在关闭 Agent...")

async def run_agent_in_jupyter():
    print("正在尝试以 Jupyter/IPython 兼容模式启动 Agent (全流程重试增强版)...")
    print("请在下方的输入提示处输入指令。输入 '退出' 结束。")
    try: await main()
    except Exception as e:
        print(f"\n🔴 Agent 在 Jupyter 模式下运行时遇到错误: {e}")
        logger.error(f"在 Jupyter 模式下运行 Agent 时出错: {e}", exc_info=True)
    finally: print("Agent 交互已结束 (Jupyter 模式)。")

if __name__ == "__main__":
    is_jupyter_env = False
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            is_jupyter_env = True
            print("检测到 Jupyter/IPython (ZMQ) 环境。")
            print("请在 Notebook cell 中执行 `await run_agent_in_jupyter()` 来启动 Agent 交互。")
        elif shell == 'TerminalInteractiveShell':
            print("检测到终端 IPython 环境。将尝试作为标准脚本启动...")
            is_jupyter_env = False 
        else:
            print(f"检测到非典型的 IPython shell ({shell})。将尝试作为标准脚本启动...")
            is_jupyter_env = False
    except NameError: is_jupyter_env = False
    if not is_jupyter_env:
        print("正在以标准 Python 脚本模式启动 Agent (全流程重试增强版)...")
        try: asyncio.run(main())
        except KeyboardInterrupt: print("\n程序被用户强制退出 (KeyboardInterrupt)。"); logger.info("[Main Script] 程序被 KeyboardInterrupt 中断。")
        except Exception as e:
            print(f"\n程序因顶层错误而意外退出: {e}")
            logger.critical(f"脚本执行期间发生顶层异常: {e}", exc_info=True)
        finally: print("Agent 程序已关闭。")

In [ ]:
await main()